<a href="https://colab.research.google.com/github/Nand0000000000/Etfs_comparison_pipeline/blob/main/bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das bibliotecas necessárias
!pip install requests pandas plotly dash yfinance schedule python-dotenv
!pip install --upgrade plotly

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import time
import os
from typing import Dict, List, Optional
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf

# Configurações globais
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Ambiente configurado com sucesso!")
print("Pipeline de Análise Financeira - Big Data")
print("=" * 50)

Ambiente configurado com sucesso!
Pipeline de Análise Financeira - Big Data


In [ ]:
class FinancialDataIngestion:
    def __init__(self):
        self.data_storage = {}
        self.apis_status = {}

    def get_nasdaq_data(self, symbol: str = "^IXIC", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Nasdaq usando Yahoo Finance
        try:
            print(f"Coletando dados do Nasdaq ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'NASDAQ'

            self.apis_status['NASDAQ'] = 'SUCCESS'
            print(f"Dados do Nasdaq coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Nasdaq: {str(e)}")
            self.apis_status['NASDAQ'] = 'ERROR'
            return pd.DataFrame()

    def get_ibovespa_data(self, symbol: str = "^BVSP", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Ibovespa usando Yahoo Finance
        try:
            print(f"Coletando dados do Ibovespa ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'IBOVESPA'

            self.apis_status['IBOVESPA'] = 'SUCCESS'
            print(f"Dados do Ibovespa coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Ibovespa: {str(e)}")
            self.apis_status['IBOVESPA'] = 'ERROR'
            return pd.DataFrame()

    def get_bitcoin_data(self, symbol: str = "BTC-USD", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Bitcoin usando Yahoo Finance
        try:
            print(f"Coletando dados do Bitcoin ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'BITCOIN'

            self.apis_status['BITCOIN'] = 'SUCCESS'
            print(f"Dados do Bitcoin coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Bitcoin: {str(e)}")
            self.apis_status['BITCOIN'] = 'ERROR'
            return pd.DataFrame()

    def collect_all_data(self) -> pd.DataFrame:
        # Coleta dados de todos os ativos
        print("=" * 50)

        all_data = []

        # Coletar dados de cada ativo
        nasdaq_data = self.get_nasdaq_data()
        if not nasdaq_data.empty:
            all_data.append(nasdaq_data)

        ibovespa_data = self.get_ibovespa_data()
        if not ibovespa_data.empty:
            all_data.append(ibovespa_data)

        bitcoin_data = self.get_bitcoin_data()
        if not bitcoin_data.empty:
            all_data.append(bitcoin_data)

        # Combinar todos os dados
        if all_data:
            combined_data = pd.concat(all_data, ignore_index=True)
            print(f"\nTotal de dados coletados: {len(combined_data)} registros")
            return combined_data
        else:
            print("Nenhum dado foi coletado com sucesso")
            return pd.DataFrame()

ingestion = FinancialDataIngestion()

In [ ]:
class FinancialDataTransformation:
    # Classe responsável pela transformação e limpeza dos dados financeiros

    def __init__(self):
        self.processed_data = None

    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:

        clean_df = df.copy()


        clean_df['Date'] = pd.to_datetime(clean_df['Date'], utc=True)


        initial_rows = len(clean_df)
        clean_df = clean_df.dropna()
        removed_rows = initial_rows - len(clean_df)


        clean_df = clean_df.sort_values(['Asset', 'Date']).reset_index(drop=True)

        print(f"Limpeza concluída:")
        print(f"   - Linhas removidas (NaN): {removed_rows}")
        print(f"   - Linhas finais: {len(clean_df)}")

        return clean_df

    def calculate_returns(self, df: pd.DataFrame) -> pd.DataFrame:
        # Calcula retornos diários, semanais e mensais


        df_returns = df.copy()

        # Calcular retornos para cada ativo
        for asset in df_returns['Asset'].unique():
            mask = df_returns['Asset'] == asset
            asset_data = df_returns[mask].copy()

            # Retorno diário
            asset_data['Daily_Return'] = asset_data['Close'].pct_change()

            # Retorno acumulado
            asset_data['Cumulative_Return'] = (1 + asset_data['Daily_Return']).cumprod() - 1

            # Volatilidade móvel (7 dias)
            asset_data['Volatility_7d'] = asset_data['Daily_Return'].rolling(window=7).std()

            # Média móvel (7 dias)
            asset_data['MA_7'] = asset_data['Close'].rolling(window=7).mean()

            # Atualizar dados no DataFrame principal
            df_returns.loc[mask, 'Daily_Return'] = asset_data['Daily_Return']
            df_returns.loc[mask, 'Cumulative_Return'] = asset_data['Cumulative_Return']
            df_returns.loc[mask, 'Volatility_7d'] = asset_data['Volatility_7d']
            df_returns.loc[mask, 'MA_7'] = asset_data['MA_7']

        print("Cálculo de retornos concluído")
        return df_returns

    def aggregate_by_period(self, df: pd.DataFrame) -> pd.DataFrame:
        # Agrega dados por períodos (semanal, mensal)
        print("Agregando dados por período...")

        # Adicionar colunas de período
        df['Year_Month'] = df['Date'].dt.to_period('M')
        df['Year_Week'] = df['Date'].dt.to_period('W')

        # Agregação mensal
        monthly_agg = df.groupby(['Asset', 'Year_Month']).agg({
            'Open': 'first',
            'Close': 'last',
            'High': 'max',
            'Low': 'min',
            'Volume': 'sum',
            'Daily_Return': 'sum'  # Retorno mensal
        }).reset_index()

        monthly_agg['Period_Type'] = 'Monthly'
        monthly_agg['Period'] = monthly_agg['Year_Month'].astype(str)

        print("Agregação concluída")
        return monthly_agg


    def process_pipeline(self, raw_data: pd.DataFrame) -> pd.DataFrame:
        # Executa todo o pipeline de transformação
        print("INICIANDO PIPELINE DE TRANSFORMAÇÃO")
        print("=" * 50)

        # 1. Limpeza
        clean_data = self.clean_data(raw_data)

        # 2. Cálculo de retornos
        data_with_returns = self.calculate_returns(clean_data)

        # 3. Agregação
        monthly_data = self.aggregate_by_period(data_with_returns)

        # Armazenar dados processados
        self.processed_data = data_with_returns

        print("\nPipeline de transformação concluído!")
        return data_with_returns

# Criar instância da classe
transformation = FinancialDataTransformation()

In [ ]:
class FinancialDataStorage:

    # Classe responsável pelo armazenamento dos dados processados


    def __init__(self):
        self.storage_path = "/content/financial_data/"
        self.create_storage_structure()

    def create_storage_structure(self):
        import os

        folders = [
            self.storage_path,
            f"{self.storage_path}raw/",
            f"{self.storage_path}processed/",
            f"{self.storage_path}aggregated/"
        ]

        for folder in folders:
            os.makedirs(folder, exist_ok=True)

        print("Estrutura de armazenamento criada")

    def save_raw_data(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"raw_data_{timestamp}.csv"

        filepath = f"{self.storage_path}raw/{filename}"
        data.to_csv(filepath, index=False)

        print(f"Dados brutos salvos: {filepath}")
        return filepath

    def save_processed_data(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"processed_data_{timestamp}.csv"

        filepath = f"{self.storage_path}processed/{filename}"
        data.to_csv(filepath, index=False)

        print(f"Dados processados salvos: {filepath}")
        return filepath

    def save_to_parquet(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"financial_data_{timestamp}.parquet"

        filepath = f"{self.storage_path}processed/{filename}"
        data.to_parquet(filepath, index=False)

        print(f"Dados salvos em Parquet (Data Lake): {filepath}")
        return filepath

    def get_storage_summary(self) -> Dict:
        import os

        summary = {
            "raw_files": len(os.listdir(f"{self.storage_path}raw/")) if os.path.exists(f"{self.storage_path}raw/") else 0,
            "processed_files": len(os.listdir(f"{self.storage_path}processed/")) if os.path.exists(f"{self.storage_path}processed/") else 0,
            "total_size_mb": 0
        }

        for root, dirs, files in os.walk(self.storage_path):
            for file in files:
                file_path = os.path.join(root, file)
                summary["total_size_mb"] += os.path.getsize(file_path)

        summary["total_size_mb"] = round(summary["total_size_mb"] / (1024 * 1024), 2)

        return summary

storage = FinancialDataStorage()

Estrutura de armazenamento criada


In [ ]:
class FinancialDashboard:

    def __init__(self, data: pd.DataFrame):
        self.data = data

    def plot_price_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(go.Scatter(
                x=asset_data['Date'],
                y=asset_data['Close'],
                mode='lines',
                name=asset,
                line=dict(width=2)
            ))

        fig.update_layout(
            title="Comparação de Preços dos Ativos",
            xaxis_title="Data",
            yaxis_title="Preço de Fechamento",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_returns_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(go.Scatter(
                x=asset_data['Date'],
                y=asset_data['Cumulative_Return'] * 100,
                mode='lines',
                name=f"{asset} - Retorno Acumulado",
                line=dict(width=2)
            ))

        fig.update_layout(
            title="Comparação de Retornos Acumulados",
            xaxis_title="Data",
            yaxis_title="Retorno Acumulado (%)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_volatility_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset].dropna(subset=['Volatility_7d'])

            if not asset_data.empty:
                fig.add_trace(go.Scatter(
                    x=asset_data['Date'],
                    y=asset_data['Volatility_7d'] * 100,
                    mode='lines',
                    name=f"{asset} - Volatilidade 7d",
                    line=dict(width=2)
                ))

        fig.update_layout(
            title="Comparação de Volatilidade (7 dias)",
            xaxis_title="Data",
            yaxis_title="Volatilidade (%)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_volume_comparison(self) -> go.Figure:
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=tuple(f"Volume - {asset}" for asset in self.data['Asset'].unique()),
            vertical_spacing=0.1
        )

        for i, asset in enumerate(self.data['Asset'].unique(), 1):
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(
                go.Bar(
                    x=asset_data['Date'],
                    y=asset_data['Volume'],
                    name=f"{asset} Volume",
                    showlegend=False
                ),
                row=i, col=1
            )

        fig.update_layout(
            title="Comparação de Volume de Negociação",
            template="plotly_white",
            height=800
        )

        return fig

    def plot_moving_average_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset].dropna(subset=['MA_7'])

            if not asset_data.empty:
                fig.add_trace(go.Scatter(
                    x=asset_data['Date'],
                    y=asset_data['MA_7'],
                    mode='lines',
                    name=f"{asset} - MA 7d",
                    line=dict(width=2)
                ))

        fig.update_layout(
            title="Comparação de Médias Móveis (7 dias)",
            xaxis_title="Data",
            yaxis_title="Média Móvel de Fechamento (7 dias)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_candlestick_chart(self, asset_name: str) -> go.Figure:
        asset_data = self.data[self.data['Asset'] == asset_name]

        if asset_data.empty:
            print(f"Dados não encontrados para o ativo: {asset_name}")
            return go.Figure()

        fig = go.Figure(data=[go.Candlestick(
            x=asset_data['Date'],
            open=asset_data['Open'],
            high=asset_data['High'],
            low=asset_data['Low'],
            close=asset_data['Close']
        )])

        fig.update_layout(
            title=f"Gráfico de Candlestick - {asset_name}",
            xaxis_title="Data",
            yaxis_title="Preço",
            template="plotly_white",
            height=500
        )

        return fig


    def create_summary_table(self) -> pd.DataFrame:
        summary_data = []

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            summary_data.append({
                'Ativo': asset,
                'Período': f"{asset_data['Date'].min().date()} a {asset_data['Date'].max().date()}",
                'Preço Inicial': f"${asset_data['Close'].iloc[0]:,.2f}",
                'Preço Final': f"${asset_data['Close'].iloc[-1]:,.2f}",
                'Retorno Total (%)': f"{(asset_data['Cumulative_Return'].iloc[-1] * 100):.2f}%",
                'Retorno Médio Diário (%)': f"{(asset_data['Daily_Return'].mean() * 100):.3f}%",
                'Volatilidade (%)': f"{(asset_data['Daily_Return'].std() * 100):.3f}%",
                'Volume Médio': f"{asset_data['Volume'].mean():,.0f}"
            })

        return pd.DataFrame(summary_data)

# Criar instância da classe
if 'processed_data' in locals() and not processed_data.empty:
    dashboard = FinancialDashboard(processed_data)
else:
    print("Não há dados processados para criar o dashboard.")

In [ ]:
raw_data = ingestion.collect_all_data()


if not raw_data.empty:
    print(f"\nResumo dos dados coletados:")
    print(f"Total de registros: {len(raw_data)}")
    print(f"Período: {raw_data['Date'].min().date()} a {raw_data['Date'].max().date()}")
    print(f"Ativos: {raw_data['Asset'].unique().tolist()}")

    print(f"\nPrimeiras linhas dos dados:")
    display(raw_data.head())


else:
    print("Nenhum dado foi coletado.")

Coletando dados do Nasdaq (^IXIC)...
Dados do Nasdaq coletados: 22 registros
Coletando dados do Ibovespa (^BVSP)...
Dados do Ibovespa coletados: 23 registros
Coletando dados do Bitcoin (BTC-USD)...
Dados do Bitcoin coletados: 31 registros

Total de dados coletados: 76 registros

Resumo dos dados coletados:
Total de registros: 76
Período: 2025-09-10 a 2025-10-12
Ativos: ['NASDAQ', 'IBOVESPA', 'BITCOIN']

Primeiras linhas dos dados:


,Date,Open,High,Low,Close,Volume,Asset
0,2025-09-11 00:00:00-04:00,21977.519531,22059.710938,21908.810547,22043.070312,10025020000,NASDAQ
1,2025-09-12 00:00:00-04:00,22078.630859,22182.339844,22031.619141,22141.099609,9023140000,NASDAQ
2,2025-09-15 00:00:00-04:00,22243.199219,22352.250000,22233.000000,22348.750000,8904030000,NASDAQ
3,2025-09-16 00:00:00-04:00,22397.500000,22397.500000,22308.779297,22333.960938,8592240000,NASDAQ
4,2025-09-17 00:00:00-04:00,22333.019531,22339.539062,22058.919922,22261.330078,9325980000,NASDAQ


In [ ]:
# Executar transformação dos dados
if not raw_data.empty:
    processed_data = transformation.process_pipeline(raw_data)

    # Mostrar resultados
    print(f"\nResumo dos dados processados:")
    print(f"Total de registros: {len(processed_data)}")
    print(f"Colunas adicionadas: Daily_Return, Cumulative_Return, Volatility_7d, MA_7")

    # Estatísticas por ativo
    print(f"\nEstatísticas por ativo:")
    for asset in processed_data['Asset'].unique():
        asset_data = processed_data[processed_data['Asset'] == asset]
        avg_return = asset_data['Daily_Return'].mean() * 100
        volatility = asset_data['Daily_Return'].std() * 100

        print(f"{asset}:")
        print(f"   - Retorno médio diário: {avg_return:.3f}%")
        print(f"   - Volatilidade: {volatility:.3f}%")
        print(f"   - Registros: {len(asset_data)}")

    # Mostrar dados processados
    print(f"\nAmostra dos dados processados:")
    display(processed_data[['Date', 'Asset', 'Close', 'Daily_Return', 'Cumulative_Return']].head(10))
else:
    print("Não há dados para processar. Execute primeiro a ingestão.")

INICIANDO PIPELINE DE TRANSFORMAÇÃO
Limpeza concluída:
   - Linhas removidas (NaN): 0
   - Linhas finais: 76
Cálculo de retornos concluído
Agregando dados por período...
Agregação concluída

Pipeline de transformação concluído!

Resumo dos dados processados:
Total de registros: 76
Colunas adicionadas: Daily_Return, Cumulative_Return, Volatility_7d, MA_7

Estatísticas por ativo:
BITCOIN:
   - Retorno médio diário: 0.004%
   - Volatilidade: 2.188%
   - Registros: 31
IBOVESPA:
   - Retorno médio diário: -0.051%
   - Volatilidade: 0.686%
   - Registros: 23
NASDAQ:
   - Retorno médio diário: 0.040%
   - Volatilidade: 1.003%
   - Registros: 22

Amostra dos dados processados:


,Date,Asset,Close,Daily_Return,Cumulative_Return
0,2025-09-12 00:00:00+00:00,BITCOIN,116101.578125,NaN,NaN
1,2025-09-13 00:00:00+00:00,BITCOIN,115950.507812,-0.001301,-0.001301
2,2025-09-14 00:00:00+00:00,BITCOIN,115407.656250,-0.004682,-0.005977
3,2025-09-15 00:00:00+00:00,BITCOIN,115444.875000,0.000322,-0.005656
4,2025-09-16 00:00:00+00:00,BITCOIN,116843.187500,0.012112,0.006388
5,2025-09-17 00:00:00+00:00,BITCOIN,116468.507812,-0.003207,0.003160
6,2025-09-18 00:00:00+00:00,BITCOIN,117137.203125,0.005741,0.008920
7,2025-09-19 00:00:00+00:00,BITCOIN,115688.859375,-0.012365,-0.003555
8,2025-09-20 00:00:00+00:00,BITCOIN,115721.960938,0.000286,-0.003270
9,2025-09-21 00:00:00+00:00,BITCOIN,115306.093750,-0.003594,-0.006852


In [ ]:
# Executar armazenamento dos dados
if 'processed_data' in locals() and not processed_data.empty:

    raw_file = storage.save_raw_data(raw_data, "financial_raw_data.csv")

    processed_file = storage.save_processed_data(processed_data, "financial_processed_data.csv")

    parquet_file = storage.save_to_parquet(processed_data, "financial_data_lake.parquet")

    summary = storage.get_storage_summary()
    print(f"\nResumo do Armazenamento:")
    print(f"   - Arquivos raw: {summary['raw_files']}")
    print(f"   - Arquivos processados: {summary['processed_files']}")
    print(f"   - Tamanho total: {summary['total_size_mb']} MB")

    print(f"\nDados armazenados com sucesso!")
else:
    print("Não há dados processados para armazenar.")

Dados brutos salvos: /content/financial_data/raw/financial_raw_data.csv
Dados processados salvos: /content/financial_data/processed/financial_processed_data.csv
Dados salvos em Parquet (Data Lake): /content/financial_data/processed/financial_data_lake.parquet

Resumo do Armazenamento:
   - Arquivos raw: 1
   - Arquivos processados: 2
   - Tamanho total: 0.03 MB

Dados armazenados com sucesso!


In [ ]:
# Gerar todas as visualizações
if 'dashboard' in locals():
    # 1. Tabela Resumo
    print("Tabela Resumo dos Ativos:")
    summary_table = dashboard.create_summary_table()
    display(summary_table)

    # 2. Gráfico de Preços
    print("\nGerando gráfico de comparação de preços...")
    price_fig = dashboard.plot_price_comparison()
    price_fig.show()

    # 3. Gráfico de Retornos
    print("\nGerando gráfico de retornos acumulados...")
    returns_fig = dashboard.plot_returns_comparison()
    returns_fig.show()

    # 4. Gráfico de Volatilidade
    print("\nGerando gráfico de volatilidade...")
    volatility_fig = dashboard.plot_volatility_comparison()
    volatility_fig.show()

    # 5. Gráfico de Volume
    print("\nGerando gráfico de volume...")
    volume_fig = dashboard.plot_volume_comparison()
    volume_fig.show()

    # 6. Gráfico de Médias Móveis
    print("\nGerando gráfico de médias móveis...")
    ma_fig = dashboard.plot_moving_average_comparison()
    ma_fig.show()


    print("\nTodas as visualizações foram geradas com sucesso")
else:
    print("Dashboard não está disponível. Execute primeiro as células anteriores.")

Tabela Resumo dos Ativos:


,Ativo,Período,Preço Inicial,Preço Final,Retorno Total (%),Retorno Médio Diário (%),Volatilidade (%),Volume Médio
0,BITCOIN,2025-09-12 a 2025-10-12,"$116,101.58","$115,424.19",-0.58%,0.004%,2.188%,"60,497,677,116"
1,IBOVESPA,2025-09-10 a 2025-10-10,"$142,349.00","$140,680.00",-1.17%,-0.051%,0.686%,"7,832,165"
2,NASDAQ,2025-09-11 a 2025-10-10,"$22,043.07","$22,204.43",0.73%,0.040%,1.003%,"9,978,965,455"



Gerando gráfico de comparação de preços...



Gerando gráfico de retornos acumulados...



Gerando gráfico de volatilidade...



Gerando gráfico de volume...



Gerando gráfico de médias móveis...



Todas as visualizações foram geradas com sucesso


In [ ]:
# Gerar gráfico de Candlestick para o Bitcoin
if 'dashboard' in locals():
    print("\nGerando gráfico de candlestick para o Bitcoin...")
    candlestick_fig = dashboard.plot_candlestick_chart("BITCOIN")
    candlestick_fig.show()
else:
    print("Dashboard não está disponível. Execute primeiro as células anteriores.")


Gerando gráfico de candlestick para o Bitcoin...


In [ ]:
if 'processed_data' in locals() and not processed_data.empty:

    print(f"\nRESULTADOS OBTIDOS:")
    print(f"• Ativos analisados: {', '.join(processed_data['Asset'].unique())}")
    print(f"• Período: {processed_data['Date'].min().date()} a {processed_data['Date'].max().date()}")
    print(f"• Total de dados: {len(processed_data)} registros")

    # Insights rápidos
    print(f"\nINSIGHTS PRELIMINARES:")
    for asset in processed_data['Asset'].unique():
        asset_data = processed_data[processed_data['Asset'] == asset]
        total_return = asset_data['Cumulative_Return'].iloc[-1] * 100
        volatility = asset_data['Daily_Return'].std() * 100
        print(f"• {asset}: Retorno {total_return:.2f}%, Volatilidade {volatility:.2f}%")
else:
    print("Erro: Pipeline não está funcionando. Verifique as células anteriores.")


RESULTADOS OBTIDOS:
• Ativos analisados: BITCOIN, IBOVESPA, NASDAQ
• Período: 2025-09-10 a 2025-10-12
• Total de dados: 76 registros

INSIGHTS PRELIMINARES:
• BITCOIN: Retorno -0.58%, Volatilidade 2.19%
• IBOVESPA: Retorno -1.17%, Volatilidade 0.69%
• NASDAQ: Retorno 0.73%, Volatilidade 1.00%
